In [ ]:
# Import necessary packages
from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import pandas as pd

# Connect to the API
api = SentinelAPI('s5pguest', 's5pguest', 'https://scihub.copernicus.eu/dhus')

# Define the area of interest
footprint = geojson_to_wkt({
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -100.3955078125,
          37.06448131901557
        ],
        [
          -93.8671875,
          37.06448131901557
        ],
        [
          -93.8671875,
          41.70203247025623
        ],
        [
          -100.3955078125,
          41.70203247025623
        ],
        [
          -100.3955078125,
          37.06448131901557
        ]
      ]
    ]
  }
})

# Define the query parameters
products = api.query(footprint,
                     date=('2022-01-01', '2022-05-18'),
                     platformname='Sentinel-5',
                     processinglevel='L2',
                     producttype='L2__CH4___',
                     cloudcoverpercentage=(0, 30))

# Convert the results to a pandas dataframe
results_df = api.to_dataframe(products)

# Sort the dataframe by date and time
results_df = results_df.sort_values(['beginposition', 'endposition'])

# Download the data for the first product in the dataframe
api.download(results_df.index[0])

# Extract the methane concentration data from the downloaded file
filepath = api.get_product_odata(results_df.index[0])['title']
df = pd.read_csv(filepath, delimiter=';', header=0, usecols=[0,1,2,3,4,5,6,7,8])
methane_data = df['methane_mixing_ratio']

# Print the first 10 values of methane data
print(methane_data[:10])
